# 29_Join_in_pyspark

Write a pyspark code perform below function 
* 52. Get employee name, project name order by firstname from "EmployeeDetail" and "ProjectDetail" for all employee even they have not assigned project. 
* 53 Get employee name, project name order by firstname from "EmployeeDetail" and "ProjectDetail" for all employee if project is not assigned then display "-No Project Assigned".

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [0]:
data = [ [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"], [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"], [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"], [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"], [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"], ]

In [0]:
schema = StructType([ StructField("EmployeeID", IntegerType(), True), StructField("First_Name", StringType(), True), StructField("Last_Name", StringType(), True), StructField("Salary", DoubleType(), True), StructField("Joining_Date", StringType(), True), StructField("Department", StringType(), True), StructField("Gender", StringType(), True) ])
pro_schema = StructType([ StructField("Project_DetailID", IntegerType(), True), StructField("Employee_DetailID", IntegerType(), True), StructField("Project_Name", StringType(), True) ])

In [0]:
pro_data = [ (1, 1, "Task Track"), (2, 1, "CLP"), (3, 1, "Survey Management"), (4, 2, "HR Management"), (5, 3, "Task Track"), (6, 3, "GRS"), (7, 3, "DDS"), (8, 4, "HR Management"), (9, 6, "GL Management") ]

In [0]:
df_emp = spark.createDataFrame(data, schema)
df_pro = spark.createDataFrame(pro_data, pro_schema)

df_emp.display()
df_pro.display()

EmployeeID,First_Name,Last_Name,Salary,Joining_Date,Department,Gender
1,Vikas,Ahlawat,600000.0,2013-02-15 11:16:28.290,IT,Male
2,nikita,Jain,530000.0,2014-01-09 17:31:07.793,HR,Female
3,Ashish,Kumar,1000000.0,2014-01-09 10:05:07.793,IT,Male
4,Nikhil,Sharma,480000.0,2014-01-09 09:00:07.793,HR,Male
5,anish,kadian,500000.0,2014-01-09 09:31:07.793,Payroll,Male


Project_DetailID,Employee_DetailID,Project_Name
1,1,Task Track
2,1,CLP
3,1,Survey Management
4,2,HR Management
5,3,Task Track
6,3,GRS
7,3,DDS
8,4,HR Management
9,6,GL Management


### Get employee name, project name order by firstname from "EmployeeDetail" and "ProjectDetail" for all employee even they have not assigned project.

In [0]:
(
    df_emp.join(
        df_pro
        , df_emp['EmployeeID'] == df_pro['Employee_DetailID']
        , 'left'
    )
    .select('First_Name', 'Project_Name')
    .orderBy(lower(col('First_Name')).asc())
    .display()
)

First_Name,Project_Name
anish,null
Ashish,Task Track
Ashish,GRS
Ashish,DDS
Nikhil,HR Management
nikita,HR Management
Vikas,Task Track
Vikas,CLP
Vikas,Survey Management


### Get employee name, project name order by firstname from "EmployeeDetail" and "ProjectDetail" for all employee if project is not assigned then display "-No Project Assigned".

In [0]:
(
    df_emp.join(
        df_pro
        , df_emp['EmployeeID'] == df_pro['Employee_DetailID']
        , 'left'
    )
    .select('First_Name', 'Project_Name')
    .withColumn(
        'project_name'
        , coalesce( (col('Project_Name')), lit('Not assigned'))
    )
    .orderBy(lower(col('First_Name')).asc())
    .display()
)

First_Name,project_name
anish,Not assigned
Ashish,Task Track
Ashish,GRS
Ashish,DDS
Nikhil,HR Management
nikita,HR Management
Vikas,Task Track
Vikas,CLP
Vikas,Survey Management
